[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/observation_prediction.ipynb)

# Transduction from observation to prediction

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or
transductive inference is reasoning from observed, specific (training) cases
to specific (test) cases. In contrast, induction is reasoning from observed
training cases to general rules, which are then applied to the test cases.
The distinction is most interesting in cases where the predictions of the
transductive model are not achievable by any inductive model. Note that this
is caused by transductive inference on different test sets producing mutually
inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

> The encoder is composed of a stack of N=6 identical layers. Each layer has
two sub-layers. The first is a multi-head self-attention mechanism, and the
second is a simple, positionwise fully connected feed-forward network.
There is a residual connection around each of the two sub-layers, followed by
layer normalization.

> The decoder is also composed of a stack of N=6 identical layers. In addition
to the two sub-layers in each encoder layer, the decoder inserts a third
sub-layer, which performs multi-head attention over the output of the encoder
stack. The self-attention sub-layer in the decoder stack is modified to prevent
positions from attending to subsequent positions.  This masking, combined with
the fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions
less than i.

In [2]:
# Preparation

! pip install -U tensor2tensor
! curl -o observation_prediction.sh http://josd.github.io/eye/transduction/observation_prediction.sh
! curl -o observation_prediction.py http://josd.github.io/eye/transduction/observation_prediction.py
! curl -o __init__.py http://josd.github.io/eye/transduction/__init__.py
! curl -o sample.observation http://josd.github.io/eye/transduction/sample.observation
! chmod +x observation_prediction.sh
#% rm -fr ~/t2t_train/observation_prediction/transformer-transformer_small/
#% rm -fr ~/t2t_data/

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
Requirement already up-to-date: tensor2tensor in /usr/local/lib/python3.6/dist-packages (1.6.1)
Requirement not upgraded as not directly required: google-api-python-client in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.6.6)
Requirement not upgraded as not directly required: sympy in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.1.1)
Requirement not upgraded as not directly required: numpy in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.14.2)
Requirement not upgraded as not directly required: scipy in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.19.1)
Requirement not upgraded as not directly required: h5py in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (2.7.1)
Requirement not upgraded as not directly required: requests in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (2.18.4)
Requirement not upgraded as not dire

In [3]:
# See the observation_prediction problem

! pygmentize -g observation_prediction.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPrediction(text_problems.Text2TextProblem):
  """Transduction from observation to prediction."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    # 50% evaluation data
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 5,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 5,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(10000):
      # wind turbine size factor
      size_f

In [4]:
# See the observation_prediction script

! pygmentize -g observation_prediction.sh

#!/bin/bash
PROBLEM=observation_prediction
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data
TMP_DIR=/tmp/t2t_datagen
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=3 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=2000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample.observation \
  --decode_hparams="beam_size=3,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --wor

In [5]:
# Run the observation_prediction script

! ./observation_prediction.sh

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module content from path /
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction
INFO:tensorflow:Generating data for observation_prediction.
INFO:tensorflow:Generating vocab file: /content/t2t_data/vocab.observation_prediction.16384.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 205
INFO:tensorflow:Iteration 1
INFO:tensorflow:vocab_size = 108
INFO:tensorflow:Iteration 2
INFO:tensorflow:vocab_size = 113
INFO:tensorflow:Iteration 3
INFO:tensorflow:vocab_size = 111
INFO:tensorflow:Trying min_count 250
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 270
INFO:tensorfl

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-04-27 21:01:15.086170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-04-27 21:01:15.086620: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1344] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-04-27 21:01:15.086669: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-04-27 21:01:15.437414: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-04-27 21:01:15.437487: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-04-27 21:01:15.437517: I tensorflow/core/common_runti

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 101 into /content/t2t_train/observation_prediction/transformer-transformer_small/model.ckpt.
INFO:tensorflow:loss = 3.463091, step = 100
INFO:tensorflow:Saving checkpoints for 200 into /content/t2t_train/observation_prediction/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 1.2464528.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction-dev*
INFO:tensorflow:partition: 0 num_data_files: 5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_754_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_754_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_754_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-27-21:03:20
INFO:tensorflow:Graph was finalized.
2018-04-27 21:03:21.107852: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-04-27 21:03:21.107940: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-04-27 21:03:21.107972: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-04-27 21:03:21.107999: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-04-27 21:03:21.108221: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow device (/job:localhost/replica:0/task:0

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/3]
INFO:tensorflow:Evaluation [2/3]
INFO:tensorflow:Evaluation [3/3]
INFO:tensorflow:Finished evaluation at 2018-04-27-21:04:08
INFO:tensorflow:Saving dict for global step 400: global_step = 400, loss = 0.64293, metrics-observation_prediction/accuracy = 0.89458275, metrics-observation_prediction/accuracy_per_sequence = 0.2217687, metrics-observation_prediction/accuracy_top5 = 0.92862374, metrics-observation_prediction/approx_bleu_score = 0.5552461, metrics-observation_prediction/neg_log_perplexity = -0.6060839, metrics-observation_prediction/rouge_2_fscore = 0.67045045, metrics-observation_prediction/rouge_L_fscore = 0.7820518
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction-train*
INFO:tensorflow:partition: 0 num_data_files: 5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variab

...

INFO:tensorflow:Graph was finalized.
2018-04-27 21:13:12.507911: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-04-27 21:13:12.508003: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-04-27 21:13:12.508043: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-04-27 21:13:12.508068: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-04-27 21:13:12.508319: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 8579 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /content/t2t_train/observation_prediction/transformer-transformer_small/model.ckpt-1600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1

INFO:tensorflow:Evaluation [1/3]
INFO:tensorflow:Evaluation [2/3]
INFO:tensorflow:Evaluation [3/3]
INFO:tensorflow:Finished evaluation at 2018-04-27-21:13:59
INFO:tensorflow:Saving dict for global step 1700: global_step = 1700, loss = 0.06256604, metrics-observation_prediction/accuracy = 0.98938507, metrics-observation_prediction/accuracy_per_sequence = 0.9265306, metrics-observation_prediction/accuracy_top5 = 0.9972547, metrics-observation_prediction/approx_bleu_score = 0.8376961, metrics-observation_prediction/neg_log_perplexity = -0.10631669, metrics-observation_prediction/rouge_2_fscore = 0.884211, metrics-observation_prediction/rouge_L_fscore = 0.87158316
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction-train*
INFO:tensorflow:partition: 0 num_data_files: 5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INF

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_754_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_754_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_754_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3882496
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-04-27 21:14:54.727044: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-04-27 21:14:54.727138: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-04-27 21:14:54.727177: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-04-27 21:14:54.727203: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-04-27 21:14:54.

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1901 into /content/t2t_train/observation_prediction/transformer-transformer_small/model.ckpt.
INFO:tensorflow:loss = 0.03809699, step = 1900
INFO:tensorflow:Saving checkpoints for 2000 into /content/t2t_train/observation_prediction/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 0.034685604.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction-dev*
INFO:tensorflow:partition: 0 num_data_files: 5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using v

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:BEAM 0:
INFO:tensorflow:Inference results INPUT: A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
INFO:tensorflow:Inference results OUTPUT: A_TURBINE producing 9533 kW
INFO:tensorflow:BEAM 1:
INFO:tensorflow:Inference results INPUT: A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
INFO:tensorflow:Inference results OUTPUT: A_TURBINE producing 3645 kW
INFO:tensorflow:BEAM 2:
INFO:tensorflow:Inference results INPUT: A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
INFO:tensorflow:Inference results OUTPUT: A_TURBINE producing 14299 kW
INFO:tensorflow:BEAM 0:
INFO:tensorflow:Inference results INPUT: A_PERSON with weight 74 kg and height 179 cm
INFO:tensorflow:Inference results OUTPUT: A_PERSON has BMI class N
INFO:tensorflow:BEAM 1:
INFO:tensorflow:Inference results INPUT: A_PERSON with weight 74 kg and height 179 cm
INFO:tensorflow:Inference result

In [0]:
# See the transductions
# For each target the top 3 is shown with their scores (log probability)

! pygmentize -g sample.observation
print("->-")
! pygmentize -g sample.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N	-0.34	A_PERSON has BMI class U	-4.58	A_PERSON has BMI class O	-5.07
A_TURBINE producing 9533 kW	-0.36	A_TURBINE producing 16682 kW	-3.11	A_TURBINE producing 16208 kW	-5.05
